In [37]:
# DSC540
# Weeks 7 & 8
# Milestone 3
# Author: Nathanael Ochoa
# 05/05/2024

# Cleaning/Formatting Website Data

I had to change the website link I was using because the previous source used iframes. The following link, https://ggweather.com/normals/monthly91.htm, will now be my website source. The page that contains the data I will be scraping can be found [here](https://ggweather.com/normals/ccd_precip.html#W).

Please take note that I am *not* claiming the data as my own but am using it for educational purposes. The links to the data source can be found above.

In [39]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [40]:
# Website variable that keeps everything neat
url = "https://ggweather.com/normals/ccd_precip.html#W"

In [41]:
# Function that extracts website tables and return a list of data frames
def extractWebTables(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        tables = soup.find_all("table")
        dataframes = []  # List of data frames
        for table in tables:
            # Convert HTML table to DataFrame
            df = pd.read_html(str(table))[0]
            dataframes.append(df)
        return dataframes
    else:
        print("Failed to request content from: {}".format(url))
        return None

In [42]:
# Function call and store returned list into 'tables'
tables = extractWebTables(url)

In [43]:
# For loop that displays the returned tables
for idx, table in enumerate(tables):
    print("DataFrame for Table {}:\n{}\n".format(idx + 1, table))

DataFrame for Table 1:
                                                   0   1   2
0  PRECIPITATION Normal (1991-2020) Source: Natio... NaN NaN

DataFrame for Table 2:
                     Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0                           NaN        NaN        NaN        NaN        NaN   
1                        Alaska        NaN        NaN        NaN        NaN   
2                      Location      State       Lat.      Long.      Elev.   
3                        Alpine         AK     70.346   -150.928       5.20   
4                       Alyeska         AK     60.958   -149.111      82.90   
...                         ...        ...        ...        ...        ...   
15394          Yellowstone Lake         WY     44.544   -110.421    2388.10   
15395      Yellowstone Np E Ent         WY     44.488   -110.004    2119.60   
15396  Yellowstone Park Mammoth         WY     44.976   -110.696    1888.20   
15397                 Yoder 5 W         W

The second table is the data that I need. I'll save this to a new variable and check that it's a data frame:

In [44]:
# Save to new variable
data = tables[1]

In [45]:
# Type check
type(data)

pandas.core.frame.DataFrame

In [46]:
# Preview data
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Location,State,Lat.,Long.,Elev.,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
3,Alpine,AK,70.346,-150.928,5.20,0.34,0.51,0.38,0.40,0.21,0.78,1.40,1.37,0.79,0.84,0.53,0.57,8.12
4,Alyeska,AK,60.958,-149.111,82.90,7.34,6.57,5.37,5.25,3.99,2.21,2.66,4.86,8.26,7.78,7.65,8.98,70.92


## Step 1 - Drop NaN values

The function retrieved all of the data from the website page but I only need the data from Washington so I can drop all of the rows that contain NaN. The data wasn't extracted like a typical data table but I can drop the NaN values without any issues.

In [47]:
# Drop NaN values
data.dropna(inplace = True)

In [48]:
# Preview changes
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
2,Location,State,Lat.,Long.,Elev.,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
3,Alpine,AK,70.346,-150.928,5.20,0.34,0.51,0.38,0.40,0.21,0.78,1.40,1.37,0.79,0.84,0.53,0.57,8.12
4,Alyeska,AK,60.958,-149.111,82.90,7.34,6.57,5.37,5.25,3.99,2.21,2.66,4.86,8.26,7.78,7.65,8.98,70.92
5,Amber Lake,AK,62.192,-150.503,136.90,1.66,1.69,1.19,1.43,2.09,1.80,3.99,4.94,5.74,2.93,2.40,2.07,31.93
6,American Creek,AK,64.790,-141.230,320.00,0.66,0.47,0.41,0.49,0.95,1.97,2.94,2.65,1.31,1.17,0.71,0.58,14.31


## Step 2 - Replace column headers

With the NaN values removed, the first row of the data frame contains the column headers. The next step will be to replace the unnamed column headers with the first row.

In [49]:
# View first row
data.iloc[0]

Unnamed: 0     Location
Unnamed: 1        State
Unnamed: 2         Lat.
Unnamed: 3        Long.
Unnamed: 4        Elev.
Unnamed: 5          Jan
Unnamed: 6          Feb
Unnamed: 7          Mar
Unnamed: 8          Apr
Unnamed: 9          May
Unnamed: 10         Jun
Unnamed: 11         Jul
Unnamed: 12         Aug
Unnamed: 13         Sep
Unnamed: 14         Oct
Unnamed: 15         Nov
Unnamed: 16         Dec
Unnamed: 17       Total
Name: 2, dtype: object

In [50]:
# Rename the columns using the first row
data.columns = data.iloc[0]

In [51]:
# Preview changes
data.head()

2,Location,State,Lat.,Long.,Elev.,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
2,Location,State,Lat.,Long.,Elev.,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
3,Alpine,AK,70.346,-150.928,5.20,0.34,0.51,0.38,0.40,0.21,0.78,1.40,1.37,0.79,0.84,0.53,0.57,8.12
4,Alyeska,AK,60.958,-149.111,82.90,7.34,6.57,5.37,5.25,3.99,2.21,2.66,4.86,8.26,7.78,7.65,8.98,70.92
5,Amber Lake,AK,62.192,-150.503,136.90,1.66,1.69,1.19,1.43,2.09,1.80,3.99,4.94,5.74,2.93,2.40,2.07,31.93
6,American Creek,AK,64.790,-141.230,320.00,0.66,0.47,0.41,0.49,0.95,1.97,2.94,2.65,1.31,1.17,0.71,0.58,14.31


## Step 3 - Drop rows that do not contain 'WA' under the *State* column

Seattle is in Washington so this is the only state that matters in my analysis. The rows that do not contain the correct state will be dropped.

In [52]:
# Calculate row count before changes
rows_before = len(data)

In [53]:
# Drop unneeded columns
data = data.query("State == 'WA'")

In [54]:
# Compare row counts after changes
print("Row count before changes: {}".format(rows_before))
print("Row count after changes:  {}".format(len(data)))

Row count before changes: 14910
Row count after changes:  336


# Step 4 - Rename column headers

I prefer having the column headers in all lowercase because it's easier to type. Additionally, I'll expand any abbreviated names for readability.

In [55]:
# Convert columns to lowercase
data.columns = [col.lower() for col in data.columns]

In [56]:
# Expand abbreviated names
data.rename(columns = {"lat.": "latitude", "long.": "longitude", "elev.": "elevation"}, inplace = True)

C:\Users\natha\AppData\Local\Temp\ipykernel_21468\452017881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {"lat.": "latitude", "long.": "longitude", "elev.": "elevation"}, inplace = True)


In [57]:
# Preview changes
data.head()

,location,state,latitude,longitude,elevation,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total
14287,Aberdeen 20Nne,WA,47.261,-123.715,132.60,21.16,13.36,15.13,9.43,4.96,4.11,2.08,2.60,5.70,13.04,20.84,21.45,133.86
14288,Alpine Meadows,WA,47.780,-121.700,1066.80,23.33,15.72,18.23,14.06,9.44,7.44,3.33,3.75,7.46,16.13,23.04,20.93,162.86
14289,Anacortes,WA,48.511,-122.614,6.10,3.93,2.58,2.42,2.06,1.86,1.50,0.76,1.38,1.55,2.78,4.36,3.78,28.96
14290,Anacortes 4.9 S,WA,48.420,-122.614,12.20,3.83,2.59,2.19,2.18,2.05,1.31,0.85,0.91,1.33,2.67,4.06,3.48,27.45
14291,Arlington,WA,48.200,-122.128,30.50,5.92,4.42,5.00,4.34,3.43,2.66,1.15,1.34,2.49,4.94,7.15,6.15,48.99


# Step 5 - Reset data frame index

The index is still using the previous one, so it needs to be reset.

In [58]:
data.reset_index(drop = True, inplace = True)

In [59]:
# Preview changes
data

,location,state,latitude,longitude,elevation,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total
0,Aberdeen 20Nne,WA,47.261,-123.715,132.60,21.16,13.36,15.13,9.43,4.96,4.11,2.08,2.60,5.70,13.04,20.84,21.45,133.86
1,Alpine Meadows,WA,47.780,-121.700,1066.80,23.33,15.72,18.23,14.06,9.44,7.44,3.33,3.75,7.46,16.13,23.04,20.93,162.86
2,Anacortes,WA,48.511,-122.614,6.10,3.93,2.58,2.42,2.06,1.86,1.50,0.76,1.38,1.55,2.78,4.36,3.78,28.96
3,Anacortes 4.9 S,WA,48.420,-122.614,12.20,3.83,2.59,2.19,2.18,2.05,1.31,0.85,0.91,1.33,2.67,4.06,3.48,27.45
4,Arlington,WA,48.200,-122.128,30.50,5.92,4.42,5.00,4.34,3.43,2.66,1.15,1.34,2.49,4.94,7.15,6.15,48.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,Winthrop 3.5 Wnw,WA,48.494,-120.248,872.60,2.59,1.85,1.65,0.93,1.34,1.15,0.73,0.70,0.75,1.35,2.46,3.14,18.64
332,Yacolt 9.0 N,WA,45.996,-122.399,139.60,15.78,10.85,11.41,9.05,5.45,3.81,0.84,1.14,3.76,9.11,15.14,15.76,102.10
333,Yakima #2,WA,46.586,-120.568,348.10,1.16,0.92,0.68,0.62,0.79,0.61,0.25,0.21,0.33,0.72,0.90,1.41,8.60
334,Yakima 2.5 Wnw,WA,46.609,-120.575,411.20,1.36,1.27,0.77,0.64,1.32,0.61,0.32,0.31,0.34,0.77,1.01,1.52,10.24


# Step 6 - Check data frame column types

The columns should be the correct types but it's always a good idea to check them. I'll make changes if any are needed.

In [60]:
# View data types
data.dtypes

location     object
state        object
latitude     object
longitude    object
elevation    object
jan          object
feb          object
mar          object
apr          object
may          object
jun          object
jul          object
aug          object
sep          object
oct          object
nov          object
dec          object
total        object
dtype: object

In [61]:
# Create a dictionary of the column names and their conversions
conversion = {"location": str, "state": str, "latitude": float, "longitude": float, "elevation": float, "jan": float, 
              "feb": float, "mar": float, "apr": float, "may": float, "jun": float, "jul": float, "aug": float, "sep": float, 
              "oct": float, "nov": float, "dec": float, "total": float}

In [62]:
# Convert the column data types
data = data.astype(conversion)

In [63]:
# Check data types
data.dtypes

location      object
state         object
latitude     float64
longitude    float64
elevation    float64
jan          float64
feb          float64
mar          float64
apr          float64
may          float64
jun          float64
jul          float64
aug          float64
sep          float64
oct          float64
nov          float64
dec          float64
total        float64
dtype: object

# Step 7 - Check for NaN values

There may have been some errors in my steps, so it's wise to check for NaN values when I believe I've finished.

In [64]:
# For loop to quickly check if there are any NaN values in any of the columns
for column in data.columns:
    print("The '{}' column contains NaN - {}".format(column, data[column].isnull().values.any()))

The 'location' column contains NaN - False
The 'state' column contains NaN - False
The 'latitude' column contains NaN - False
The 'longitude' column contains NaN - False
The 'elevation' column contains NaN - False
The 'jan' column contains NaN - False
The 'feb' column contains NaN - False
The 'mar' column contains NaN - False
The 'apr' column contains NaN - False
The 'may' column contains NaN - False
The 'jun' column contains NaN - False
The 'jul' column contains NaN - False
The 'aug' column contains NaN - False
The 'sep' column contains NaN - False
The 'oct' column contains NaN - False
The 'nov' column contains NaN - False
The 'dec' column contains NaN - False
The 'total' column contains NaN - False


Now for a final preview of the dataset:

In [65]:
data.head(10)

,location,state,latitude,longitude,elevation,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total
0,Aberdeen 20Nne,WA,47.261,-123.715,132.6,21.16,13.36,15.13,9.43,4.96,4.11,2.08,2.60,5.70,13.04,20.84,21.45,133.86
1,Alpine Meadows,WA,47.780,-121.700,1066.8,23.33,15.72,18.23,14.06,9.44,7.44,3.33,3.75,7.46,16.13,23.04,20.93,162.86
2,Anacortes,WA,48.511,-122.614,6.1,3.93,2.58,2.42,2.06,1.86,1.50,0.76,1.38,1.55,2.78,4.36,3.78,28.96
3,Anacortes 4.9 S,WA,48.420,-122.614,12.2,3.83,2.59,2.19,2.18,2.05,1.31,0.85,0.91,1.33,2.67,4.06,3.48,27.45
4,Arlington,WA,48.200,-122.128,30.5,5.92,4.42,5.00,4.34,3.43,2.66,1.15,1.34,2.49,4.94,7.15,6.15,48.99
5,Asotin 14 Sw,WA,46.203,-117.247,1037.2,0.79,1.11,1.60,1.72,2.30,1.94,0.85,0.67,0.69,1.13,1.17,1.27,15.24
6,Auburn 8.2 Se,WA,47.264,-122.067,170.1,7.29,4.41,5.17,4.60,3.44,2.90,1.20,1.40,2.17,4.62,6.52,6.25,49.97
7,Baring,WA,47.772,-121.482,234.7,16.72,10.60,11.90,8.35,5.02,3.96,1.73,1.90,4.79,12.30,18.06,14.56,109.89
8,Battle Ground,WA,45.771,-122.529,86.6,7.20,5.32,5.59,4.54,3.36,2.31,0.63,0.80,2.20,4.82,7.61,7.99,52.37
9,Beaver Pass,WA,48.880,-121.260,1106.4,11.74,7.16,8.57,3.83,2.48,2.09,1.23,1.68,3.70,7.72,13.45,12.19,75.84


I ensured to include both a disclaimer and a link to the data's source. I only used web scraping techniques to extract the data from the HTML code. The website specifies the source of all of the data is the [National Center for Environmental Information](https://www.ncei.noaa.gov/). The data wasn't initially in a typical table format, but rectifying this was no problem. I dropped empty values and kept data that was from the state of Washington. Additionally, I adjusted the column headers for better comprehension. Importantly, I made no alterations to the data itself that would raise any ethical concerns.

In [66]:
# Download as CSV to use in Milestone 5
data.to_csv("M5_html_table.csv")